# COMS 4995_002 Deep Learning Assignment 2

In [110]:
%matplotlib inline
from datetime import datetime
from enum import Enum
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import tensorflow as tf
import glob
import sys
import time

In [111]:
# Helper functions, DO NOT modify this

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [112]:
# Functions to load data, DO NOT change these

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    y_one_hot[y] = 1
    return y_one_hot.T

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)

## Create the model

In [116]:
# When Using L2 Regularizaton on weights, contant to use:
REG_LAMBDA_L2 = 0.01
DROPOUT_RATE = 0.25

class Optimizer(Enum):
    ADAM = 1
    SGD = 2

OPTIMIZER = Optimizer.ADAM
PLOT_ACTIVATIONS = False

def Basic_CNN1(features, labels, mode):
    """
    Defines BasicCNN1 architecture.
    This model roughly parallels the simple architecture in the Tensorflow 
    examples code, provided with Tensorflow Python library:
    ...\tensorflow-r1.3\tensorflow\examples\tutorials\mnist
    
    But with the setup modified for CIFAR10, 
    and various other modifications for performance.
    """
    
    #[batch_size, width, height, channels]
    # CIFAR10 are 32 x 32 x 3
    input_layer = tf.reshape(features["x"], [-1, 32, 32, 3])

    # First convolution layer 
    conv0 = tf.layers.conv2d(inputs=input_layer, 
                             filters=32,
                             kernel_size=[3,3],
                             strides=(1,1),
                             padding="same",
                             activation=tf.nn.relu,
                             kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False), # Use Gaussian
                             kernel_regularizer=tf.contrib.layers.l2_regularizer(REG_LAMBDA_L2),
                             activity_regularizer=None,
                             dilation_rate=(1, 1),
                             name='conv0')
    
    # Dropout using DROPOUT_RATE probability
    dropout0 = tf.layers.dropout(inputs=conv0, 
                                 rate=DROPOUT_RATE, 
                                 training=mode==tf.estimator.ModeKeys.TRAIN, 
                                 name='dropout0')
    # Pooling
    pool0 = tf.layers.max_pooling2d(inputs=dropout0, pool_size=[2, 2], strides=2, name='pool0')
    
    # Batch norm
    batchnorm0 = tf.contrib.layers.batch_norm(inputs=pool0,
                                              center=True,
                                              scale=True,
                                              activation_fn=tf.nn.relu,
                                              is_training=mode==tf.estimator.ModeKeys.TRAIN)


    # Second convolution layer
    conv1 = tf.layers.conv2d(inputs=batchnorm0, filters=32,
                             kernel_size=[3,3],
                             padding="same",
                             activation=tf.nn.relu,
                             name='conv1')

    # Dropout using DROPOUT_RATE probability
    dropout1 = tf.layers.dropout(inputs=conv1, 
                                 rate=DROPOUT_RATE, 
                                 training=mode==tf.estimator.ModeKeys.TRAIN, 
                                 name='dropout1')
    
    # Pooling Layer #1
    # Max pooling, 2x2 filter, stride 2
    pool1 = tf.layers.max_pooling2d(inputs=dropout1, pool_size=[2, 2], strides=2, name='pool1')

    batchnorm1 = tf.contrib.layers.batch_norm(inputs=pool1,
                                              center=True,
                                              scale=True,
                                              activation_fn=tf.nn.relu,
                                              is_training=mode==tf.estimator.ModeKeys.TRAIN)

    # Third convolution layer
    conv2 = tf.layers.conv2d(inputs=batchnorm1, 
                             filters=16,
                             kernel_size=[1,1],
                             strides=(1,1),
                             padding="same",
                             activation=tf.nn.relu,
                             kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False), # Use Gaussian
                             kernel_regularizer=tf.contrib.layers.l2_regularizer(REG_LAMBDA_L2),
                             activity_regularizer=None,
                             dilation_rate=(1, 1),
                             name='conv2')

    # Dropout using DROPOUT_RATE probability
    dropout2 = tf.layers.dropout(inputs=conv2, 
                                 rate=DROPOUT_RATE, 
                                 training=mode==tf.estimator.ModeKeys.TRAIN, 
                                 name='dropout2')


    # Pooling Layer #2
    # Max pooling, 2x2 filter, stride 2
    pool2 = tf.layers.max_pooling2d(inputs=dropout2, pool_size=[2, 2], strides=2, name='pool2')
    
    batchnorm2 = tf.contrib.layers.batch_norm(inputs=pool2,
                                              center=True,
                                              scale=True,
                                              activation_fn=tf.nn.relu,
                                              is_training= mode==tf.estimator.ModeKeys.TRAIN)
    
    # Flatten batchnorm in to input into dense layer
    batchnorm2_flat = tf.contrib.layers.flatten(batchnorm2)
    
    # Dense Layer
    dense = tf.layers.dense(inputs=batchnorm2_flat, units=512, activation=tf.nn.relu, name='dense')
    
    # Logits layer
    logits = tf.layers.dense(inputs=dense, units=10, name='logits')

    # Generate predictions (for PREDICT and EVAL mode)
    predictions = {"classes": tf.argmax(input=logits, axis=1), 
                   "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
                   }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)


    # Loss (for both TRAIN and EVAL modes)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels,
                                           logits=logits)
    reg_loss = sum(tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES))
    loss += reg_loss
    
    # Add evaluation metrics (for TRAIN and EVAL modes)
    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=labels,
                                                       predictions=predictions["classes"])}

    # Training (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        trainables = tf.trainable_variables()
        if OPTIMIZER == Optimizer.SGD:
            optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        elif OPTIMIZER == Optimizer.ADAM:
            optimizer = tf.train.AdamOptimizer()

        train_op = optimizer.minimize(loss=loss,
                                      global_step=tf.train.get_global_step())
        # Set up summary for logging training loss and accuracy
        tf.summary.scalar('train__loss', loss)
        tf.summary.scalar('train__accuracy', eval_metric_ops['accuracy'][1])
        if OPTIMIZER == Optimizer.SGD:
            tf.summary.scalar('train__learning_rate', optimizer._learning_rate)
        elif OPTIMIZER == Optimizer.ADAM:
            tf.summary.scalar('train__learning_rate', optimizer._lr)

                
        if PLOT_ACTIVATIONS:
            # Make histograms of ACTIVATIONS
            tf.summary.histogram('conv0__activations',conv0)
            tf.summary.histogram('conv1filter1__activations',conv1[:,:,:,0])
            tf.summary.histogram('conv1__activations',conv1)
    
            tf.summary.histogram('pool1__activations',pool1)
            tf.summary.histogram('conv2__activations',conv2)
            tf.summary.histogram('pool2__activations',pool2)
            tf.summary.histogram('dense1__activations',dense1)

        # Make histograms of WEIGHTS
        for i, Variable in enumerate(trainables):
            tf.summary.histogram(Variable.name.replace(':', '_'), Variable)
        
        # Make histograms of GRADIENTS:
        for i, Variable in enumerate(trainables):
            #if Variable.name.startswith('dense1') or Variable.name.startswith('logits'):
            #    tf.summary.histogram(Variable.name+'__grad',optimizer.compute_gradients(loss, Variable))
            #Above if want only last dense layers. Below if want all layers [inc. batchnorm etc.]
            tf.summary.histogram(Variable.name.replace(':', '_') + '__grad',
                                 optimizer.compute_gradients(loss, Variable))

        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op, eval_metric_ops=eval_metric_ops)

    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

## Load the data, using utility functions from HW1.

In [114]:
# Load the data, using utility functions from HW1
data_root_path = 'cifar10-hw1/'
X, y = get_train_data(data_root_path) # this may take a few minutes
X_test = get_images(data_root_path + 'test')
print('Data loading done')

# Hold out 10% of the data to use as a validation set:
train_inds = np.random.choice(X.shape[1],int(X.shape[1]*.90),replace=False)
validation_inds = np.setdiff1d(np.arange(X.shape[1]),train_inds)
X_train = np.moveaxis(X[:,train_inds].reshape(32,32,3,train_inds.size),-1,0).astype(np.float32)
y_train = y[train_inds].astype(np.int32)
X_validation = np.moveaxis(X[:,validation_inds].reshape(32,32,3,validation_inds.size),-1,0).astype(np.float32)
y_validation = y[validation_inds].astype(np.int32)

{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


## Train, test

In [118]:
MAX_STEPS = 1000 # 10000
TIMESTAMP = datetime.now().isoformat(timespec='minutes') # '2017-10-28T13:13'

output_dir = "/tmp/cifar10_CNN/{}".format(TIMESTAMP)
print("Writing output to {}".format(output_dir))

start_time = time.time()

# Create the Estimator (using our own model)
cifar10_classifier = tf.estimator.Estimator(model_fn=Basic_CNN1,
                                            model_dir=output_dir)


# Set up logging for predictions
# Log the values in the "Softmax" tensor with label "probabilities"
tensors_to_log = {} # {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log,
                                          every_n_iter=50)

# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": X_train},
                                                    y=y_train,
                                                    batch_size=100,
                                                    num_epochs=None,
                                                    shuffle=False)
cifar10_classifier.train(input_fn=train_input_fn,
                         steps=MAX_STEPS,
                         hooks=[logging_hook])
tf.train.SummarySaverHook(save_steps=50, output_dir=output_dir,
                          summary_op=tf.summary.merge_all)


#Test on validation set:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": X_validation},
                                                   y=y_validation,
                                                   num_epochs=1,
                                                   shuffle=False)
eval_results = cifar10_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)
print("Done in {} minutes.".format((time.time() - start_time)/60.0))

#    #Make predictions on Test set, and save them out:
#    X_test...
#    #SAVE
#    print('saving')
#    y_predicted3 = aaaaa#cifar10_classifier.evaluate(input_fn=eval_input_fn)
#    save_predictions('ans3-uni',y_predicted3)
#    
#    #CHECK LOADED
#    # test if your numpy file has been saved correctly
#    loaded_y3 = np.load('ans3-uni.npy')
#    print(loaded_y3.shape)
#    loaded_y3[:10]

Writing output to /tmp/cifar10_CNN/2017-10-30T10:12
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/cifar10_CNN/2017-10-30T10:12', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/cifar10_CNN/2017-10-30T10:12/model.ckpt.
INFO:tensorflow:
INFO:tensorflow:loss = 2.6168, step = 1
INFO:tensorflow: (13.927 sec)
INFO:tensorflow:global_step/sec: 3.53451
INFO:tensorflow: (14.371 sec)
INFO:tensorflow:loss = 1.79053, step = 101 (28.299 sec)
INFO:tensorflow: (14.239 sec)
INFO:tensorflow:global_step/sec: 3.48023
INFO:tensorflow: (14.495 sec)
INFO:tensorflow:loss = 1.65499, step = 201 (28.735 sec)
INFO:tensorflow: (13.706 sec)
INFO:tensorflow:global_step/sec: 3.58711
INFO:tensorfl